In [1]:
import pandas as pd
import numpy as np

In [7]:
import gc

In [2]:
reserve_tb = pd.read_csv('./data/reserve.csv', encoding='UTF-8')

In [3]:
hotel_tb = pd.read_csv('./data/hotel.csv', encoding='UTF-8')

In [44]:
customer_tb = pd.read_csv('./data/customer.csv', encoding='UTF-8')

## 4. 결합
데이터 테이블 결합

### 4.1. 마스터 테이블에서 정보 얻기

In [6]:
# not good
pd.merge(
    reserve_tb,
    hotel_tb,
    on='hotel_id',
    how='inner'
).query(
    'people_num == 1 & is_business'
)

# good
pd.merge(
    reserve_tb.query('people_num == 1'),
    hotel_tb.query('is_business'),
    on='hotel_id',
    how='inner'
)

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,base_price,big_area_name,small_area_name,hotel_latitude,hotel_longitude,is_business
0,r7,h_256,c_1,2017-12-29 10:38:36,2018-01-25,10:30:00,2018-01-28,1,103500,34500,C,C-1,38.237294,140.696131,True
1,r997,h_256,c_244,2016-10-15 22:47:40,2016-10-31,10:30:00,2016-11-02,1,69000,34500,C,C-1,38.237294,140.696131,True
2,r2602,h_256,c_650,2016-05-10 00:42:56,2016-05-12,11:00:00,2016-05-14,1,69000,34500,C,C-1,38.237294,140.696131,True
3,r3738,h_256,c_930,2017-04-12 09:53:00,2017-05-08,11:30:00,2017-05-09,1,34500,34500,C,C-1,38.237294,140.696131,True
4,r11,h_183,c_2,2016-11-19 12:49:10,2016-12-08,11:00:00,2016-12-11,1,29700,9900,G,G-4,33.595248,130.633567,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
441,r2766,h_56,c_698,2017-04-08 16:41:45,2017-05-03,12:30:00,2017-05-06,1,95700,31900,C,C-3,38.435640,140.898354,True
442,r2983,h_56,c_753,2016-06-18 23:17:50,2016-07-13,10:00:00,2016-07-14,1,31900,31900,C,C-3,38.435640,140.898354,True
443,r2839,h_231,c_715,2016-04-06 02:51:01,2016-04-08,09:00:00,2016-04-10,1,29200,14600,A,A-3,35.914151,139.837520,True
444,r3476,h_43,c_866,2016-09-17 21:09:02,2016-10-03,09:30:00,2016-10-04,1,18100,18100,B,B-1,35.437093,139.799077,True


### 4.2. 조건에 따라 결합할 마스터 테이블 변경

In [16]:
small_area_mst = hotel_tb \
    .groupby(['big_area_name', 'small_area_name'], as_index=False) \
    .size() \
    .reset_index()
small_area_mst.columns=['big_area_name', 'small_area_name', 'hotel_cnt']

small_area_mst['join_area_id'] = \
    np.where(small_area_mst['hotel_cnt'] - 1 >= 20,
             small_area_mst['small_area_name'],
             small_area_mst['big_area_name'])
small_area_mst.drop(['hotel_cnt', 'big_area_name'], axis=1, inplace=True)

base_hotel_mst = pd.merge(
    hotel_tb,
    small_area_mst,
    on='small_area_name'
).loc[:, ['hotel_id', 'join_area_id']]

del small_area_mst
gc.collect()

base_hotel_mst

,hotel_id,join_area_id
0,h_1,D
1,h_79,D
2,h_125,D
3,h_127,D
4,h_129,D
...,...,...
295,h_137,F
296,h_163,F
297,h_172,F
298,h_174,F


In [21]:
recommend_hotel_mst = pd.concat([
    hotel_tb[['small_area_name', 'hotel_id']] \
        .rename(columns={'small_area_name': 'join_area_id'}, inplace=False),
    hotel_tb[['big_area_name', 'hotel_id']] \
        .rename(columns={'big_area_name': 'join_area_id'}, inplace=False)
])

recommend_hotel_mst.rename(columns={'hotel_id': 'rec_hotel_id'}, inplace=True)

result = pd.merge(
    base_hotel_mst,
    recommend_hotel_mst,
    on='join_area_id'
).loc[:, ['hotel_id', 'rec_hotel_id']] \
.query('hotel_id != rec_hotel_id')

result

,hotel_id,rec_hotel_id
1,h_1,h_14
2,h_1,h_22
3,h_1,h_27
4,h_1,h_40
5,h_1,h_45
...,...,...
9659,h_220,h_226
9660,h_220,h_249
9661,h_220,h_271
9662,h_220,h_281


### 4.3. 과거 데이터에서 정보 얻기

In [24]:
result = reserve_tb \
    .groupby('customer_id') \
    .apply(lambda group:
           group.sort_values(by='reserve_datetime', axis=0, inplace=False))

# 이전 데이터
result['before_price'] = \
    result['total_price'].groupby('customer_id').shift(periods=2)

result[['customer_id', 'reserve_datetime', 'total_price', 'before_price']]

customer_id     reserve_datetime  total_price  before_price
customer_id                                                                 
c_1         0            c_1  2016-03-06 13:09:42        97200           NaN
            1            c_1  2016-07-16 23:39:55        20600           NaN
            2            c_1  2016-09-24 10:03:17        33600       97200.0
            3            c_1  2017-03-08 03:20:10       194400       20600.0
            4            c_1  2017-09-05 19:50:37        68100       33600.0
...                      ...                  ...          ...           ...
c_999       4023       c_999  2016-10-06 18:01:34        59400           NaN
            4024       c_999  2017-03-11 11:56:05        37200       62400.0
            4025       c_999  2017-06-27 23:00:02        16000       59400.0
            4026       c_999  2017-09-29 05:24:57        41800       37200.0
            4027       c_999  2018-03-14 05:01:45        74800       16000.0

[4030 rows x 4 columns]

In [28]:
result = reserve_tb.groupby('customer_id') \
    .apply(lambda x: x.sort_values(by='reserve_datetime', ascending=True)) \
    .reset_index(drop=True)
    
# 현재와 이전 데이터의 합
result['price_sum'] = pd.Series(
    result.loc[:, ['customer_id', 'total_price']]
    .groupby('customer_id')
    .rolling(center=False, window=3, min_periods=3).sum()
    .reset_index(drop=True)
    .loc[:, 'total_price']
)

result[['customer_id', 'reserve_datetime', 'total_price', 'price_sum']]

,customer_id,reserve_datetime,total_price,price_sum
0,c_1,2016-03-06 13:09:42,97200,NaN
1,c_1,2016-07-16 23:39:55,20600,NaN
2,c_1,2016-09-24 10:03:17,33600,151400.0
3,c_1,2017-03-08 03:20:10,194400,248600.0
4,c_1,2017-09-05 19:50:37,68100,296100.0
...,...,...,...,...
4025,c_999,2016-10-06 18:01:34,59400,NaN
4026,c_999,2017-03-11 11:56:05,37200,159000.0
4027,c_999,2017-06-27 23:00:02,16000,112600.0
4028,c_999,2017-09-29 05:24:57,41800,95000.0


In [30]:
result = reserve_tb.groupby('customer_id') \
    .apply(lambda x: x.sort_values(by='reserve_datetime', ascending=True)) \
    .reset_index(drop=True)

# 이전 데이터들의 평균

result['price_avg'] = pd.Series(
    result
    .groupby('customer_id')
    ['total_price']
    .rolling(center=False, window=3, min_periods=1).mean()
    .reset_index(drop=True)
)

result['price_avg'] = \
    result.groupby('customer_id')['price_avg'].shift(periods=1)

result[['customer_id', 'reserve_datetime', 'total_price', 'price_avg']]

,customer_id,reserve_datetime,total_price,price_avg
0,c_1,2016-03-06 13:09:42,97200,NaN
1,c_1,2016-07-16 23:39:55,20600,97200.000000
2,c_1,2016-09-24 10:03:17,33600,58900.000000
3,c_1,2017-03-08 03:20:10,194400,50466.666667
4,c_1,2017-09-05 19:50:37,68100,82866.666667
...,...,...,...,...
4025,c_999,2016-10-06 18:01:34,59400,62400.000000
4026,c_999,2017-03-11 11:56:05,37200,60900.000000
4027,c_999,2017-06-27 23:00:02,16000,53000.000000
4028,c_999,2017-09-29 05:24:57,41800,37533.333333


In [41]:
import pandas.tseries.offsets as offsets
import operator

reserve_tb['reserve_datetime'] = \
    pd.to_datetime(reserve_tb['reserve_datetime'], format='%Y-%m-%d %H:%M:%S')

sum_table = pd.merge(
    reserve_tb[['reserve_id', 'customer_id', 'reserve_datetime']],
    reserve_tb[['customer_id', 'reserve_datetime', 'total_price']]
        .rename(columns={'reserve_datetime': 'reserve_datetime_before'}),
    on='customer_id'
)

sum_table = \
    sum_table[operator.and_(
        sum_table['reserve_datetime'] > sum_table['reserve_datetime_before'],
        sum_table['reserve_datetime'] + offsets.Day(-90) <=
            sum_table['reserve_datetime_before']
    )] \
    .groupby('reserve_id') \
    ['total_price'] \
    .sum() \
    .reset_index()

sum_table.columns = ['reserve_id', 'total_price_sum']

result = pd.merge(reserve_tb, sum_table, on='reserve_id', how='left').fillna(0)

result[['reserve_id', 'reserve_datetime', 'total_price', 'total_price_sum']]

,reserve_id,reserve_datetime,total_price,total_price_sum
0,r1,2016-03-06 13:09:42,97200,0.0
1,r2,2016-07-16 23:39:55,20600,0.0
2,r3,2016-09-24 10:03:17,33600,20600.0
3,r4,2017-03-08 03:20:10,194400,0.0
4,r5,2017-09-05 19:50:37,68100,0.0
...,...,...,...,...
4025,r4026,2017-06-27 23:00:02,16000,0.0
4026,r4027,2017-09-29 05:24:57,41800,0.0
4027,r4028,2018-03-14 05:01:45,74800,0.0
4028,r4029,2016-04-16 15:20:17,540000,0.0


### 4.4. 상호 결합

In [52]:
import datetime
from dateutil.relativedelta import relativedelta

month_mst = pd.DataFrame({
    'year_month': [
        (datetime.date(2017, 1, 1) + relativedelta(months=x))
            .strftime("%Y%m") for x in range(0, 3)
    ]
})

print(month_mst)

customer_tb['join_key'] = 0
month_mst['join_key'] = 0

customer_mst = pd.merge(
    customer_tb[['customer_id', 'join_key']],
    month_mst,
    on='join_key'
)

print(customer_mst)

reserve_tb['year_month'] = reserve_tb['checkin_date'] \
    .apply(lambda x: pd.to_datetime(x, format='%Y-%m-%d').strftime('%Y%m'))

summary_result = pd.merge(
    customer_mst,
    reserve_tb[['customer_id', 'year_month', 'total_price']],
    on=['customer_id', 'year_month'],
    how='left'
).groupby(['customer_id', 'year_month']) \
['total_price'] \
.sum() \
.reset_index()

summary_result.fillna(0, inplace=True)
summary_result

  year_month
0     201701
1     201702
2     201703
     customer_id  join_key year_month
0            c_1         0     201701
1            c_1         0     201702
2            c_1         0     201703
3            c_2         0     201701
4            c_2         0     201702
...          ...       ...        ...
2995       c_999         0     201702
2996       c_999         0     201703
2997      c_1000         0     201701
2998      c_1000         0     201702
2999      c_1000         0     201703

[3000 rows x 3 columns]


,customer_id,year_month,total_price
0,c_1,201701,0.0
1,c_1,201702,0.0
2,c_1,201703,194400.0
3,c_10,201701,0.0
4,c_10,201702,0.0
...,...,...,...
2995,c_998,201702,0.0
2996,c_998,201703,0.0
2997,c_999,201701,0.0
2998,c_999,201702,0.0
